In [2]:
import os
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-11.0.13"
os.environ["SPARK_HOME"] = "C:\Spark\spark-3.2.0-bin-hadoop3.2"
os.environ["HADOOP_HOME"] = "C:\Spark\spark-3.2.0-bin-hadoop3.2"
#os.environ["PYSPARK_PYTHON"] = "C:\Users\Liu\anaconda3\python.exe"

In [3]:
import os
print(os.environ['SPARK_HOME'])
print(os.environ['JAVA_HOME'])
print(os.environ['PATH'])

C:\Spark\spark-3.2.0-bin-hadoop3.2
C:\Program Files\Java\jdk-11.0.13
C:\Users\Liu\anaconda3\envs\PySpark_DS;C:\Users\Liu\anaconda3\envs\PySpark_DS\Library\mingw-w64\bin;C:\Users\Liu\anaconda3\envs\PySpark_DS\Library\usr\bin;C:\Users\Liu\anaconda3\envs\PySpark_DS\Library\bin;C:\Users\Liu\anaconda3\envs\PySpark_DS\Scripts;C:\Users\Liu\anaconda3\envs\PySpark_DS\bin;C:\Users\Liu\anaconda3\condabin;C:\Program Files\Common Files\Oracle\Java\javapath;C:\Program Files (x86)\Common Files\Oracle\Java\javapath;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0;C:\Windows\System32\OpenSSH;C:\Program Files\Microsoft VS Code\bin;C:\Program Files\Git\cmd;C:\spark\spark-3.1.2-bin-hadoop3.2\bin;C:\Program Files\Java\jre1.8.0_301\bin;C:\Program Files\Java\jdk-11.0.13;C:\Program Files\Microsoft SQL Server\120\Tools\Binn;C:\Program Files\Common Files\Autodesk Shared;C:\Program Files\Microsoft SQL Server\Client SDK\ODBC\170\Tools\Binn;C:\Program Files (x86)\M

In [ ]:
!pip install -q findspark

In [4]:
import findspark
findspark.init("C:\Spark\spark-3.2.0-bin-hadoop3.2")


In [5]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

conf = SparkConf().set('spark.ui.port','4050')
sc = SparkContext(conf=conf,appName="ExeApp")
#ss = SparkSession.builder.master('localhost[*]').getOrCreate()
ss = SparkSession.builder.master('local[*]').getOrCreate()
spark = ss

## Trabalho

Com os arquivos contidos em /Dados/ETrabalho descritos abaixo:

* cities.csv
    - id
* lines.csv
    - city_id
* station_lines.csv
    - line_id
    - station_id
* station.csv
    - city_id

#### Responder:
* Qual país possui cidades?
* Qual cidade possui linhas?
* Quais são 10 linhas que passam por mais cidades?
* Qual é o país que tem mais linhas?
* A linha que tem mais estações em seu percurso?
* Quais são as 10 cidades que possuem as linhas com mais estações?
* Qual país são os mais presentes entre essas 10 cidades?
* O país que tem a maior média de estações?



In [51]:
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, DateType
from pyspark.sql import functions as F
sqlc = SQLContext(sc)

In [52]:
schema = StructType([
    StructField("id", IntegerType(), True ),
    StructField("name", StringType(), True),
    StructField("coords", StringType(), True),
    StructField("start_year", IntegerType(), True ),
    StructField("url_name", StringType(), True),
    StructField("country", StringType(), True),
    StructField("country_state", StringType(), True)])

df = sqlc.read.csv("Dados/Etrabalho/cities.csv",schema, header=True)
df.createOrReplaceTempView("cities")

In [53]:
schema2 = StructType([
    StructField("id", IntegerType(), True ),
    StructField("city_id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("url_name", StringType(), True ),
    StructField("color", StringType(), True),
    StructField("system_id", IntegerType(), True),
    StructField("transport_mode_id", IntegerType(), True)])

df2 = sqlc.read.csv("Dados/Etrabalho/lines.csv", schema2, header=True)
df2.createOrReplaceTempView("lines")

In [54]:
schema3 = StructType([
    StructField("id", IntegerType(), True ),
    StructField("name", StringType(), True),
    StructField("geometry", StringType(), True),
    StructField("buildstart", IntegerType(), True ),
    StructField("opening", IntegerType(), True),
    StructField("closure", IntegerType(), True),
    StructField("city_id", IntegerType(), True)])

df3 = sqlc.read.csv("Dados/Etrabalho/stations.csv", schema3, header=True)
df3.createOrReplaceTempView("stations")

In [55]:
schema4 = StructType([
    StructField("id", IntegerType(), True ),
    StructField("station_id", IntegerType(), True),
    StructField("line_id", IntegerType(), True),
    StructField("city_id", IntegerType(), True ),
    StructField("created_at", DateType(), True),
    StructField("updated_at", DateType(), True)])

df4 = sqlc.read.csv("Dados/Etrabalho/station_lines.csv", schema4, header=True)
df4.createOrReplaceTempView("station_lines")

In [56]:
# Qual país possui cidades?
ss.sql('select country, count(name) as contagem from cities \
        group by country \
        order by count(name) desc').show()

+-------------+--------+
|      country|contagem|
+-------------+--------+
|United States|     115|
|       France|      71|
|       Canada|      14|
|        Spain|       8|
|      England|       8|
|    Argentina|       6|
|        China|       6|
|    Australia|       6|
|      Germany|       5|
|        Italy|       5|
|       Mexico|       5|
|        Chile|       4|
|       Brazil|       4|
|        Japan|       4|
|       Russia|       4|
| South Africa|       3|
|        India|       3|
|     Scotland|       3|
|      Austria|       3|
|       Norway|       2|
+-------------+--------+
only showing top 20 rows



In [57]:
# Qual cidade possui linhas?
ss.sql('select c.id, c.name, count(l.name) as contagem \
        from lines as l \
        left join cities as c on c.id = l.city_id \
        group by c.id, c.name \
        order by count(l.name) desc').show()

+---+--------------+--------+
| id|          name|contagem|
+---+--------------+--------+
|114|         Tokyo|     253|
| 91|         Osaka|     112|
| 95|         Paris|      75|
|206|      New York|      70|
|106|     São Paulo|      40|
| 69|        London|      35|
|  4|      Santiago|      34|
|124|    Valparaíso|      31|
| 77|     Melbourne|      31|
| 82|        Munich|      29|
|  1|  Buenos Aires|      28|
| 15|       Beijing|      27|
|139|        Boston|      27|
|265|        Nantes|      26|
|107|      Shanghai|      24|
|102|Rio de Janeiro|      23|
| 78|   Mexico City|      22|
| 27|      Brussels|      22|
| 14|     Barcelona|      19|
| 79|         Milan|      18|
+---+--------------+--------+
only showing top 20 rows



In [58]:
# Quais são 10 linhas que passam por mais cidades?
ss.sql('select l.id, l.name, count(c.id) as cidade \
        from lines as l \
        left join cities as c on c.id = l.city_id \
        group by l.id, l.name \
        order by count(c.id) asc').show(10)

+----+--------------------+------+
|  id|                name|cidade|
+----+--------------------+------+
| 275|Linha 6 • Laranja...|     1|
| 850|         Barrie Line|     1|
|  40|            Línea 4A|     1|
|1481|    Kobe Kosoku Line|     1|
| 506|        T 11 express|     1|
| 687|      Northern Lines|     1|
| 641|        Hondori Line|     1|
| 736|Disneyland Resort...|     1|
|1104|            del Bajo|     1|
|1262|        Jamaica Line|     1|
+----+--------------------+------+
only showing top 10 rows



In [59]:
# Qual é o país que tem mais linhas?

ss.sql('select c.country, count(l.name) as linhas \
        from lines as l \
        left join cities as c on c.id = l.city_id \
        group by c.country \
        order by count(l.name) desc').show()

+-------------+------+
|      country|linhas|
+-------------+------+
|        Japan|   365|
|       France|   202|
|United States|   142|
|        Chile|    70|
|       Brazil|    65|
|        China|    62|
|        Spain|    50|
|    Australia|    45|
|      Germany|    40|
|      England|    40|
|        Italy|    32|
|    Argentina|    28|
|       Canada|    27|
|       Mexico|    22|
|      Belgium|    22|
|      Austria|    20|
|     Scotland|    20|
|     Colombia|    16|
|       Sweden|    12|
|      Bolivia|    11|
+-------------+------+
only showing top 20 rows



In [60]:
# A linha que tem mais estações em seu percurso?

ss.sql('select l.id, l.name, count(distinct sl.station_id) as contagem \
        from station_lines as sl \
        left join lines as l on l.id = sl.line_id \
        left join stations as s on s.id = sl.station_id \
        group by l.id, l.name \
        order by count(distinct sl.station_id) desc \
        ').show()

+----+--------------------+--------+
|  id|                name|contagem|
+----+--------------------+--------+
| 658|                Roca|      95|
| 639|               Mitre|      90|
| 612|                   A|      87|
| 171|        Southeastern|      86|
| 938|                  17|      78|
| 316|             Ligne 1|      75|
| 613|                   B|      74|
| 367|                 T 1|      72|
| 946|                  19|      70|
| 940|                  16|      66|
|1103|               Norte|      66|
| 305|             Linie 1|      62|
| 950|                   A|      62|
| 139|District Line - U...|      62|
|1198|                   1|      60|
| 590|                   1|      60|
| 967|                   A|      59|
| 317|             Ligne 2|      59|
|1253|                   4|      59|
| 172|            Southern|      58|
+----+--------------------+--------+
only showing top 20 rows



In [61]:
# Quais são as 10 cidades que possuem as linhas com mais estações? v1
ss.sql('select  c.name, max(t0.contagem)\
        from lines as l \
        left join cities as c on c.id = l.city_id \
        left join (\
            select sl.line_id, count(distinct sl.station_id) as contagem \
            from station_lines as sl \
            group by sl.line_id) t0 on t0.line_id = l.id \
        group by c.name \
        order by max(t0.contagem) desc \
        ').show(10)

+------------+-------------+
|        name|max(contagem)|
+------------+-------------+
|Buenos Aires|           95|
|    Bordeaux|           87|
|      London|           86|
|      Munich|           78|
|      Nantes|           75|
|       Paris|           72|
|       Tours|           62|
|        Graz|           62|
| Montpellier|           60|
|   Innsbruck|           60|
+------------+-------------+
only showing top 10 rows



In [62]:
# Quais são as 10 cidades que possuem as linhas com mais estações? v2

ss.sql('select c.name, l.name, count(distinct station_id) as contagem \
        from station_lines as sl \
        left join lines as l on l.id = sl.line_id \
        left join stations as s on s.id = sl.station_id \
        left join cities as c on c.id = sl.city_id \
        left join (\
            select sl.line_id, count(distinct sl.station_id) as contagem \
            from station_lines as sl \
            group by sl.line_id) t0 on t0.line_id = sl.line_id \
        group by c.name, l.name \
        order by count(distinct station_id) desc \
        ').show(10)

+------------+------------+--------+
|        name|        name|contagem|
+------------+------------+--------+
|Buenos Aires|        Roca|      95|
|Buenos Aires|       Mitre|      90|
|    Bordeaux|           A|      87|
|      London|Southeastern|      86|
|      Munich|          17|      78|
|    New York|   Main Line|      78|
|      Nantes|     Ligne 1|      75|
|    Bordeaux|           B|      74|
|       Paris|         T 1|      72|
|      Munich|          19|      70|
+------------+------------+--------+
only showing top 10 rows



In [63]:
#Qual país são os mais presentes entre essas 10 cidades?

ss.sql('select country, name from \
                (select c.country ,c.name, max(t0.contagem) \
                from lines as l \
                left join cities as c on c.id = l.city_id \
                left join (\
                    select sl.line_id, count(distinct sl.station_id) as contagem \
                    from station_lines as sl \
                    group by sl.line_id) t0 on t0.line_id = l.id \
                group by c.country, c.name \
                order by max(t0.contagem) desc \
                limit 10) \
        order by country asc \
        ').show()

+---------+------------+
|  country|        name|
+---------+------------+
|Argentina|Buenos Aires|
|  Austria|        Graz|
|  Austria|   Innsbruck|
|  England|      London|
|   France|    Bordeaux|
|   France|      Nantes|
|   France|       Paris|
|   France|       Tours|
|   France| Montpellier|
|  Germany|      Munich|
+---------+------------+



In [64]:
# O país que tem a maior média de estações?
ss.sql('select country, avg(contagem) as media from \
                (select c.country, count(s.id) as contagem \
                from stations as s \
                left join cities as c on c.id = s.city_id \
                group by c.country) \
        group by country \
        order by avg(contagem) desc \
        ').show()

+--------------+------+
|       country| media|
+--------------+------+
|         Japan|4613.0|
|        France|2763.0|
| United States|2001.0|
|       England| 943.0|
|         China| 705.0|
|     Argentina| 685.0|
|          null| 618.0|
|        Mexico| 456.0|
|         Italy| 422.0|
|       Germany| 385.0|
|         Spain| 385.0|
|        Brazil| 381.0|
|         Chile| 306.0|
|       Austria| 306.0|
|        Canada| 218.0|
|      Scotland| 157.0|
|     Australia| 113.0|
|       Hungary|  62.0|
|      Portugal|  56.0|
|Czech Republic|  46.0|
+--------------+------+
only showing top 20 rows



# SELECTS

In [65]:
ss.sql('select * from cities').show()

+---+----------+--------------------+----------+----------+----------------+-------------+
| id|      name|              coords|start_year|  url_name|         country|country_state|
+---+----------+--------------------+----------+----------+----------------+-------------+
|  5|  Aberdeen|  POINT(-2.15 57.15)|      2017|  aberdeen|        Scotland|         null|
|  6|  Adelaide|POINT(138.6 -34.9...|      2017|  adelaide|       Australia|         null|
|  7|   Algiers|POINT(3 36.83333333)|      2017|   algiers|         Algeria|         null|
|  9|    Ankara|POINT(32.91666667...|      2017|    ankara|          Turkey|         null|
| 16|     Belém|POINT(-48.4833333...|      2017|     belem|          Brazil|         null|
| 10|  Asunción|POINT(-57.6666666...|      2017|  asuncion|        Paraguay|         null|
| 11|    Athens|POINT(23.71666667...|      2017|    athens|          Greece|         null|
| 12|  Auckland|POINT(174.75 -36....|      2017|  auckland|     New Zealand|         null|

In [66]:
ss.sql('select * from lines').show()

+----+-------+--------------------+--------------------+-------+---------+-----------------+
|  id|city_id|                name|            url_name|  color|system_id|transport_mode_id|
+----+-------+--------------------+--------------------+-------+---------+-----------------+
|  43|      4|             Línea 2|          43-linea-2|#ffbe2e|      267|                4|
|  75|     34|Línea 3 Metro de ...|75-linea-3-metro-...|   #000|      119|             null|
| 107|    126|             Línea 1|         107-linea-1|#434343|      249|             null|
| 604|     74|          La navette|      604-la-navette|#009ab9|      346|             null|
|  61|     66|  Línea 1 (Tramo 1A)|          61-linea-1|#49aa43|      250|             null|
|1471|     91|  Tokaido Shinkansen|1471-tokaido-shin...|#0000ff|      551|                1|
|   5|      1|                   A|                   a|#00b3da|      254|                4|
| 168|    261|             Línea 1|         168-linea-1|#f58223|      

In [ ]:
ss.sql('select * from stations').show()

+----+--------------------+--------------------+----------+-------+-------+-------+
|  id|                name|            geometry|buildstart|opening|closure|city_id|
+----+--------------------+--------------------+----------+-------+-------+-------+
|7694|    Keisei Tsudanuma|POINT(140.0248121...|      1921|   1921| 999999|    114|
|6003|   Kossuth Lajos tér|POINT(19.04623765...|         0|      0| 999999|     29|
|7732|       Saint-Charles|POINT(5.3801556 4...|      1973|   1977| 999999|     74|
|7695|Keisei Makuhari-H...|POINT(140.0421467...|      1991|   1991| 999999|    114|
|7726|           Chartreux|POINT(5.4014815 4...|      1973|   1977| 999999|     74|
|7696|     Keisei Makuhari|POINT(140.0560770...|      1921|   1921| 999999|    114|
|7719|            Malpassé|POINT(5.4165033 4...|      1973|   1977| 999999|     74|
|7697|            Kemigawa|POINT(140.0663045...|      1921|   1921| 999999|    114|
|   1|             Caseros|POINT(-58.3989075...|      2001|   2007|   null| 

In [ ]:
ss.sql('select * from station_lines').show()

+---+----------+-------+-------+----------+----------+
| id|station_id|line_id|city_id|created_at|updated_at|
+---+----------+-------+-------+----------+----------+
| 47|      7754|    570|     74|2017-11-21|2017-11-21|
| 48|      7771|    571|     74|2017-11-21|2017-11-21|
| 49|      7764|    571|     74|2017-11-21|2017-11-21|
| 50|      7763|    571|     74|2017-11-21|2017-11-21|
| 51|      7729|    571|     74|2017-11-21|2017-11-21|
| 52|      7772|    571|     74|2017-11-21|2017-11-21|
| 53|      8523|    582|     70|2017-11-21|2017-11-21|
| 54|      8525|    582|     70|2017-11-21|2017-11-21|
| 55|        93|     10|      1|2017-11-21|2017-11-21|
| 56|      6005|    442|    114|2017-11-21|2017-11-21|
| 57|      6102|    495|    114|2017-11-21|2017-11-21|
| 58|      6006|    439|    114|2017-11-21|2017-11-21|
| 59|      6007|    442|    114|2017-11-21|2017-11-21|
| 61|      6008|    442|    114|2017-11-21|2017-11-21|
| 62|      6009|    439|    114|2017-11-21|2017-11-21|
| 63|     